### Importing Packages

In [ ]:
import sys
sys.path.insert(0, 'C:/Users/enioh/Documents/Github/MMA-ML-Model')

from os.path import abspath, dirname, join

import pandas as pd
import numpy as np

### Importing Fully Processed Data

In [ ]:
df = pd.read_csv(abspath(join(dirname(abspath('')), 'data/', 'processed/', 'extracted_stats.csv')))

## MODEL 1: XGBoost 

In [ ]:
from src.models.XGBoost_Model import make_model
from src.models.train_model import train_model

clf = make_model()
clf, data = train_model(clf)

In [ ]:
X, y = data
y[y == 'W'] = 1
y[y != 1] = 0
y = y.to_numpy(dtype=int)

### Not that good, lets search for hyperparameters

In [5]:
from src.models.Optuna import OptunaTuning

tuner = OptunaTuning(X,y)
tuner.run()